# Install missing python modules

In [1]:
import sys
!{sys.executable} -m pip install skrebate

You should consider upgrading via the 'c:\users\miker\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.


# Global imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import datasets
from ReliefF import ReliefF
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score 
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

# Pair plot of phenotype data (=discovering pairwise correlation)

In [3]:
# growth_df=pd.read_csv('growth_SV', delimiter=" ")
# #factorize sex and population to use in PP
# growth_df['sex'] = pd.factorize(growth_df.sex)[0]
# growth_df['population'] = pd.factorize(growth_df.population)[0]
# # Slicing out the phenotype data
# data=growth_df.iloc[:, 0:9]
# # Exclude indv from pairing, by putting it as index
# data=data.set_index('indv')
# # Sex 0 = male, sex 1 = unknown and sex 2 = female
# # pop 0 = 2013B12, pop 1 = 2013B11, pop 2 = 2013B9 and pop 3 = 2013B10
# sns.pairplot(data, hue="age", palette="tab10")

# Categorising fork_length


#### Load growth data for SV's

In [2]:
growth_df=pd.read_csv('SNP_growth_50kb', delimiter=" ")  

In [4]:
growth_df

,indv,sex,population,fork_length,age,spot_count,disease,deformity,nostril_count,LG1_50000_.,...,LG25_3750000_0/2,LG25_3750000_1/2,LG25_3750000_2/2,LG25_3800000_.,LG25_3800000_0/0,LG25_3800000_0/1,LG25_3800000_1/1,LG25_3800000_0/2,LG25_3800000_1/2,LG25_3800000_2/2
0,27033489,male,2013B12,268.211388,3,53.0,2.0,0.0,1.0,1,...,11.0,1.0,6.0,3.0,1255.0,174.0,40.0,10.0,1.0,5.0
1,27033489,male,2013B12,162.068598,1,54.0,0.0,0.0,1.0,1,...,11.0,1.0,6.0,3.0,1255.0,174.0,40.0,10.0,1.0,5.0
2,83800743,unknown,2013B11,162.617027,1,39.0,0.0,0.0,2.0,9,...,14.0,3.0,3.0,1.0,1291.0,170.0,14.0,6.0,4.0,3.0
3,83800743,unknown,2013B11,264.152366,3,30.0,2.0,0.0,2.0,9,...,14.0,3.0,3.0,1.0,1291.0,170.0,14.0,6.0,4.0,3.0
4,140942470,unknown,2013B9,156.474088,1,44.0,0.0,0.0,1.0,20,...,9.0,4.0,3.0,3.0,1218.0,232.0,20.0,8.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,985661182,unknown,2013B9,149.724143,1,35.0,0.0,0.0,2.0,13,...,11.0,5.0,2.0,5.0,1294.0,158.0,14.0,9.0,5.0,1.0
59,986994303,male,2013B12,250.116536,3,40.0,2.0,0.0,1.0,23,...,9.0,2.0,5.0,1.0,1165.0,279.0,22.0,16.0,3.0,3.0
60,986994303,male,2013B12,148.071168,1,40.0,0.0,0.0,1.0,23,...,9.0,2.0,5.0,1.0,1165.0,279.0,22.0,16.0,3.0,3.0
61,998916898,unknown,2013B9,168.723524,1,49.0,0.0,0.0,1.0,3,...,6.0,6.0,3.0,1.0,1301.0,150.0,13.0,12.0,7.0,5.0


#### Split data in year 1 and year 3 data

In [3]:
growth_y1 = growth_df.loc[(growth_df['age'] == 1)]
growth_y3 = growth_df.loc[(growth_df['age'] == 3)]

#### Bin into small --> medium --> big

##### Year 1

In [4]:
small=growth_y1.loc[(growth_y1['fork_length'] >= min(growth_y1.fork_length)) & (growth_y1['fork_length'] <= growth_y1.fork_length.quantile(0.33))]
small['fork_length']="small"
small


medium=growth_y1.loc[(growth_y1['fork_length'] > growth_y1.fork_length.quantile(0.33)) & (growth_y1['fork_length'] <= growth_y1.fork_length.quantile(0.66))]
medium['fork_length']="medium"
medium

big=growth_y1.loc[growth_y1['fork_length'] > growth_y1.fork_length.quantile(0.66)]
big['fork_length']="big"
big

small_medium=small.append(medium)
binned_y1=small_medium.append(big)

<ipython-input-4-500e8c2916d0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small['fork_length']="small"
<ipython-input-4-500e8c2916d0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medium['fork_length']="medium"
<ipython-input-4-500e8c2916d0>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

##### Year 3

In [5]:
small=growth_y3.loc[(growth_y3['fork_length'] >= min(growth_y3.fork_length)) & (growth_y3['fork_length'] <= growth_y3.fork_length.quantile(0.33))]
small['fork_length']="small"
small


medium=growth_y3.loc[(growth_y3['fork_length'] > growth_y3.fork_length.quantile(0.33)) & (growth_y3['fork_length'] <= growth_y3.fork_length.quantile(0.66))]
medium['fork_length']="medium"
medium

big=growth_y3.loc[growth_y3['fork_length'] > growth_y3.fork_length.quantile(0.66)]
big['fork_length']="big"
big

small_medium=small.append(medium)
binned_y3=small_medium.append(big)

<ipython-input-5-3ebd253edfbf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small['fork_length']="small"
<ipython-input-5-3ebd253edfbf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medium['fork_length']="medium"
<ipython-input-5-3ebd253edfbf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [6]:
binned_y3['sex'] = pd.factorize(binned_y3.sex)[0]
binned_y3['population'] = pd.factorize(binned_y3.population)[0]
x=binned_y3.drop('fork_length',1)
y = binned_y3.fork_length
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5,random_state=2)

print("--- Classifiers with all features and year 3 data ---\n")
classifier = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("KNN", accuracy_score(y_test, y_pred))


classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("Naive Bayes", accuracy_score(y_test, y_pred))

clf = LogisticRegression(random_state=0).fit(x_train, y_train)
pred = clf.predict(x_test)
print("Logistic regression", accuracy_score(y_test, pred))

ranfor = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 5, max_depth = 200)
ranfor.fit(x_train, y_train)
pred_ranfor = ranfor.predict(x_test)
print("Random forest", accuracy_score(y_test, pred_ranfor))
pred_ranfor


binned_y1['sex'] = pd.factorize(binned_y1.sex)[0]
binned_y1['population'] = pd.factorize(binned_y1.population)[0]
x=binned_y1.drop('fork_length',1)
y = binned_y1.fork_length
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5,random_state=2)

print("--- Classifiers with all features and year 1 data ---\n")
classifier = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("KNN", accuracy_score(y_test, y_pred))


classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("Naive Bayes", accuracy_score(y_test, y_pred))

clf = LogisticRegression(random_state=0).fit(x_train, y_train)
pred = clf.predict(x_test)
print("Logistic regression", accuracy_score(y_test, pred))

ranfor = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 5, max_depth = 200)
ranfor.fit(x_train, y_train)
pred_ranfor = ranfor.predict(x_test)
print("Random forest", accuracy_score(y_test, pred_ranfor))
pred_ranfor

# sv = svm.SVC(kernel='linear')
# sv.fit(x_train, y_train)
# pred_svm = sv.predict(x_test)
# print("SVM",accuracy_score(y_test, pred_svm))

--- Classifiers with all features and year 3 data ---

KNN 0.375
Naive Bayes 0.4375
Logistic regression 0.375
Random forest 0.3125
--- Classifiers with all features and year 1 data ---

KNN 0.3125
Naive Bayes 0.3125
Logistic regression 0.25
Random forest 0.3125


array(['big', 'big', 'big', 'big', 'big', 'big', 'big', 'big', 'medium',
       'big', 'big', 'small', 'big', 'big', 'big', 'big'], dtype=object)

# Do relief F feature selection

## Overview of target

In [9]:
print(binned_y1['fork_length'].value_counts())
print(binned_y3['fork_length'].value_counts())

big       11
small     11
medium    10
Name: fork_length, dtype: int64
big       11
small     10
medium    10
Name: fork_length, dtype: int64


## Start with year 1

In [10]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from skrebate.turf import TuRF
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
binned_y1['sex'] = pd.factorize(binned_y1.sex)[0]
binned_y1['population'] = pd.factorize(binned_y1.population)[0]
genetic_data=binned_y1
features, labels = genetic_data.drop('fork_length', axis=1).values, genetic_data['fork_length'].values
headers = list(genetic_data.drop("fork_length", axis=1))
fs1 = TuRF(core_algorithm="ReliefF", n_features_to_select=2, pct=0.5,verbose=True)
fs1.fit(features, labels, headers)
reliefdict1={}
for feature_name, feature_score in zip(genetic_data.drop('fork_length', axis=1).columns, fs1.feature_importances_):
    reliefdict1[feature_score]=feature_name

Created distance array in 3.404171943664551 seconds.
Feature scoring under way ...
Completed scoring in 252.17766094207764 seconds.
Created distance array in 1.4953465461730957 seconds.
Feature scoring under way ...
Completed scoring in 121.7877140045166 seconds.
Created distance array in 0.7127585411071777 seconds.
Feature scoring under way ...
Completed scoring in 58.99551868438721 seconds.


## Get top 25 scores year 1

In [11]:
# print(reliefdict)
y1_relief_top=[]
for value in fs1.top_features_[0:25]:
    score=fs1.feature_importances_[value]
    y1_relief_top.append(reliefdict1[score])
    
print(y1_relief_top)

['LG11_13550000_1/1', 'LG19_20300000_.', 'LG16_11350000_1/1', 'LG21_800000_.', 'LG5_3050000_1/1', 'LG14_19200000_.', 'LG20_20650000_0/1', 'LG19_26250000_2/2', 'LG7_31450000_0/0', 'LG7_31450000_0/0', 'LG19_15250000_1/1', 'LG15_100000_.', 'LG16_17450000_0/2', 'LG20_20650000_0/0', 'LG11_15500000_.', 'LG19_20300000_0/1', 'LG11_1400000_0/0', 'LG13_30800000_2/2', 'LG16_13900000_0/1', 'LG1_1700000_0/2', 'LG1_1700000_0/2', 'LG11_13550000_0/0', 'LG13_25650000_1/1', 'LG13_8650000_.', 'LG7_25300000_1/2']


## Get prediction year 1

In [12]:
x=binned_y1[y1_relief_top]
y = binned_y1.fork_length
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5,random_state=2)

print("--- Classifiers with TuRF Relief-F top",str(len(y1_relief_top)), "features and year 1 data ---\n")
classifier = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("KNN", accuracy_score(y_test, y_pred))


classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("Naive Bayes", accuracy_score(y_test, y_pred))

clf = LogisticRegression(random_state=0).fit(x_train, y_train)
pred = clf.predict(x_test)
print("Logistic regression", accuracy_score(y_test, pred))

ranfor = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 5, max_depth = 200)
ranfor.fit(x_train, y_train)
pred_ranfor = ranfor.predict(x_test)
print("Random forest", accuracy_score(y_test, pred_ranfor))
pred_ranfor

sv = svm.SVC(kernel='linear')
sv.fit(x_train, y_train)
pred_svm = sv.predict(x_test)
print("SVM",accuracy_score(y_test, pred_svm))

--- Classifiers with TuRF Relief-F top 25 features and year 1 data ---

KNN 0.8125
Naive Bayes 0.5
Logistic regression 0.625
Random forest 0.6875
SVM 0.5625


c:\users\miker\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## Now do year 3

In [13]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from skrebate.turf import TuRF
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
binned_y3['sex'] = pd.factorize(binned_y3.sex)[0]
binned_y3['population'] = pd.factorize(binned_y3.population)[0]
genetic_data=binned_y3
features, labels = genetic_data.drop('fork_length', axis=1).values, genetic_data['fork_length'].values
headers = list(genetic_data.drop("fork_length", axis=1))
fs3 = TuRF(core_algorithm="ReliefF", n_features_to_select=2, pct=0.5,verbose=True)
fs3.fit(features, labels, headers)
reliefdict3={}
for feature_name, feature_score in zip(genetic_data.drop('fork_length', axis=1).columns, fs3.feature_importances_):
    reliefdict3[feature_score]=feature_name

Created distance array in 3.227729320526123 seconds.
Feature scoring under way ...
Completed scoring in 230.61371421813965 seconds.
Created distance array in 1.4723265171051025 seconds.
Feature scoring under way ...
Completed scoring in 112.87941217422485 seconds.
Created distance array in 0.6821603775024414 seconds.
Feature scoring under way ...
Completed scoring in 56.1529278755188 seconds.


## Get top 25 year 3

In [20]:
# print(reliefdict)
y3_relief_top=[]
for value in fs3.top_features_[0:25]:
    score=fs3.feature_importances_[value]
    y3_relief_top.append(reliefdict3[score])
    
print(y3_relief_top)

['LG1_28500000_1/2', 'LG1_28500000_2/2', 'LG20_14100000_0/0', 'LG1_28500000_1/1', 'LG12_4450000_0/0', 'LG19_9700000_0/2', 'LG8_36750000_1/1', 'LG8_33050000_0/0', 'LG8_33800000_0/0', 'LG25_3800000_2/2', 'LG7_33600000_2/2', 'LG19_9700000_2/2', 'LG21_2000000_.', 'LG19_9700000_0/1', 'LG21_25650000_.', 'LG7_27650000_2/2', 'LG8_34950000_0/2', 'LG7_11950000_1/2', 'LG10_32200000_.', 'LG10_17550000_0/1', 'LG17_20800000_1/2', 'LG20_20650000_0/1', 'LG19_14150000_.', 'LG7_25300000_1/2', 'LG17_24150000_.']


## Get prediction year 3

In [21]:
x=binned_y3[y3_relief_top]
y = binned_y3.fork_length
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5,random_state=2)

print("--- Classifiers with TuRF Relief-F top",str(len(y3_relief_top)), "features and year 3 data ---\n")
classifier = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("KNN", accuracy_score(y_test, y_pred))

classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("Naive Bayes", accuracy_score(y_test, y_pred))

clf = LogisticRegression(random_state=0,max_iter=2000).fit(x_train, y_train)
pred = clf.predict(x_test)
print("Logistic regression", accuracy_score(y_test, pred))

ranfor = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 5, max_depth = 200)
ranfor.fit(x_train, y_train)
pred_ranfor = ranfor.predict(x_test)
print("Random forest", accuracy_score(y_test, pred_ranfor))
pred_ranfor

sv = svm.SVC(kernel='linear')
sv.fit(x_train, y_train)
pred_svm = sv.predict(x_test)
print("SVM",accuracy_score(y_test, pred_svm))

--- Classifiers with TuRF Relief-F top 25 features and year 3 data ---

KNN 0.75
Naive Bayes 0.875
Logistic regression 0.6875
Random forest 0.875
SVM 0.625


# Year 1 chi-square feature selection and prediction

In [16]:
binned_y1['sex'] = pd.factorize(binned_y1.sex)[0]
binned_y1['population'] = pd.factorize(binned_y1.population)[0]
y = binned_y1.fork_length
X = binned_y1.drop(columns=["fork_length"])
bestfeatures = SelectKBest(score_func=chi2, k=10)
bestfeatures
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  
var=str(featureScores.nlargest(26,'Score'))
var.split()
data_listc1=[]
for values in var.split():
    if "LG" in values:
        data_listc1.append(values)
x=X[data_listc1]
y = binned_y1.fork_length
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5,random_state=2)


print("--- Classifiers with chi-square top",str(len(data_listc1)), "features and year 1 data ---\n")
classifier = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("KNN", accuracy_score(y_test, y_pred))


classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("Naive Bayes", accuracy_score(y_test, y_pred))

clf = LogisticRegression(random_state=0,max_iter=2000).fit(x_train, y_train)
pred = clf.predict(x_test)
print("Logistic regression", accuracy_score(y_test, pred))

ranfor = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 5, max_depth = 200)
ranfor.fit(x_train, y_train)
pred_ranfor = ranfor.predict(x_test)
print("Random forest", accuracy_score(y_test, pred_ranfor))
pred_ranfor

sv = svm.SVC(kernel='linear')
sv.fit(x_train, y_train)
pred_svm = sv.predict(x_test)
print("SVM",accuracy_score(y_test, pred_svm))

--- Classifiers with chi-square top 25 features and year 1 data ---

KNN 0.5
Naive Bayes 0.6875
Logistic regression 0.5
Random forest 0.4375
SVM 0.4375


# Year 3 chi-square feature selection and prediction

In [17]:
binned_y3['sex'] = pd.factorize(binned_y3.sex)[0]
binned_y3['population'] = pd.factorize(binned_y3.population)[0]
y = binned_y3.fork_length
X = binned_y3.drop(columns=["fork_length"])
bestfeatures = SelectKBest(score_func=chi2, k=10)
bestfeatures
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  
var=str(featureScores.nlargest(26,'Score'))
var.split()
data_listc3=[]
for values in var.split():
    if "LG" in values:
        data_listc3.append(values)
x=X[data_listc3]
y = binned_y3.fork_length
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5,random_state=2)


print("--- Classifiers with chi-square top",str(len(data_listc3)), "features and year 3 data ---\n")
classifier = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("KNN", accuracy_score(y_test, y_pred))


classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("Naive Bayes", accuracy_score(y_test, y_pred))

clf = LogisticRegression(random_state=0,max_iter=2000).fit(x_train, y_train)
pred = clf.predict(x_test)
print("Logistic regression", accuracy_score(y_test, pred))

ranfor = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 5, max_depth = 200)
ranfor.fit(x_train, y_train)
pred_ranfor = ranfor.predict(x_test)
print("Random forest", accuracy_score(y_test, pred_ranfor))
pred_ranfor

sv = svm.SVC(kernel='linear')
sv.fit(x_train, y_train)
pred_svm = sv.predict(x_test)
print("SVM",accuracy_score(y_test, pred_svm))

--- Classifiers with chi-square top 25 features and year 3 data ---

KNN 0.3125
Naive Bayes 0.5
Logistic regression 0.5625
Random forest 0.5625
SVM 0.5


# Mutual information feature selection year 1

In [18]:
binned_y1['sex'] = pd.factorize(binned_y1.sex)[0]
binned_y1['population'] = pd.factorize(binned_y1.population)[0]
y = binned_y1.fork_length
X = binned_y1.drop(columns=["fork_length"])
bestfeatures = SelectKBest(score_func=mutual_info_classif, k=10)
bestfeatures
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  
var=str(featureScores.nlargest(25,'Score'))
var.split()
data_listm1=[]
for values in var.split():
    if "LG" in values:
        data_listm1.append(values)
x=X[data_listm1]
y = binned_y1.fork_length
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5,random_state=2)


print("--- Classifiers with mutual information top",str(len(data_listm1)), "features and year 1 data ---\n")
classifier = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("KNN", accuracy_score(y_test, y_pred))


classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("Naive Bayes", accuracy_score(y_test, y_pred))

clf = LogisticRegression(random_state=0,max_iter=2000).fit(x_train, y_train)
pred = clf.predict(x_test)
print("Logistic regression", accuracy_score(y_test, pred))

ranfor = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 5, max_depth = 200)
ranfor.fit(x_train, y_train)
pred_ranfor = ranfor.predict(x_test)
print("Random forest", accuracy_score(y_test, pred_ranfor))
pred_ranfor

sv = svm.SVC(kernel='linear')
sv.fit(x_train, y_train)
pred_svm = sv.predict(x_test)
print("SVM",accuracy_score(y_test, pred_svm))

--- Classifiers with mutual information top 25 features and year 1 data ---

KNN 0.375
Naive Bayes 0.625
Logistic regression 0.625
Random forest 0.625
SVM 0.5


# Mutual information feature selection year 3

In [19]:
binned_y3['sex'] = pd.factorize(binned_y3.sex)[0]
binned_y3['population'] = pd.factorize(binned_y3.population)[0]
y = binned_y3.fork_length
X = binned_y3.drop(columns=["fork_length"])
bestfeatures = SelectKBest(score_func=mutual_info_classif, k=10)
bestfeatures
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  
var=str(featureScores.nlargest(25,'Score'))
var.split()
data_listm3=[]
for values in var.split():
    if "LG" in values:
        data_listm3.append(values)
x=X[data_listm3]
y = binned_y3.fork_length
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5,random_state=2)


print("--- Classifiers with mutual information top",str(len(data_listm3)), "features and year 3 data ---\n")
classifier = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("KNN", accuracy_score(y_test, y_pred))


classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("Naive Bayes", accuracy_score(y_test, y_pred))

clf = LogisticRegression(random_state=0,max_iter=2000).fit(x_train, y_train)
pred = clf.predict(x_test)
print("Logistic regression", accuracy_score(y_test, pred))

ranfor = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 5, max_depth = 200)
ranfor.fit(x_train, y_train)
pred_ranfor = ranfor.predict(x_test)
print("Random forest", accuracy_score(y_test, pred_ranfor))
pred_ranfor

sv = svm.SVC(kernel='linear')
sv.fit(x_train, y_train)
pred_svm = sv.predict(x_test)
print("SVM",accuracy_score(y_test, pred_svm))

--- Classifiers with mutual information top 25 features and year 3 data ---

KNN 0.4375
Naive Bayes 0.75
Logistic regression 0.5625
Random forest 1.0
SVM 0.5


In [25]:
y_test

32    medium
5      small
46       big
41       big
16    medium
59     small
6      small
0        big
49       big
51    medium
25    medium
3     medium
29     small
39     small
19     small
8        big
Name: fork_length, dtype: object